In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("yainage90/fashion-pattern-images")

Resolving data files:   0%|          | 0/1900 [00:00<?, ?it/s]

In [9]:
labels = dataset["train"].features["label"].names
print("Labels:", labels)

Labels: ['argyle', 'camouflage', 'checked', 'dot', 'floral', 'geometric', 'gradient', 'graphic', 'houndstooth', 'leopard', 'lettering', 'muji', 'paisley', 'snake_skin', 'snow_flake', 'stripe', 'tropical', 'zebra', 'zigzag']


In [11]:
from sklearn.model_selection import train_test_split

# Split the train set into train and test
split = dataset["train"].train_test_split(test_size=0.2, seed=42)
trainset = split["train"]
testset = split["test"]
print("Trainset size:", len(trainset))
print("Testset size:", len(testset))


Trainset size: 1520
Testset size: 380


In [ ]:
from datasets import load_dataset
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
import torch
from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score

# Load dataset and split
dataset = load_dataset("yainage90/fashion-pattern-images")
split = dataset["train"].train_test_split(test_size=0.2, seed=42)
trainset = split["train"]
testset = split["test"]

# Get label info
labels = trainset.features["label"].names
num_labels = len(labels)
id2label = {str(i): l for i, l in enumerate(labels)}
label2id = {l: i for i, l in enumerate(labels)}

# Preprocessing (ViT processor + optional augmentation)
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

augment = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

def transform_examples(batch):
    # Data augmentation only on training set
    if "train" in batch["__split__"][0]:
        images = [augment(img.convert("RGB")) for img in batch["image"]]
    else:
        images = [img.convert("RGB") for img in batch["image"]]
    processed = processor(images=images, return_tensors="pt")
    # Remove batch dimension for each image
    pixel_values = [img for img in processed["pixel_values"]]
    return {
        "pixel_values": pixel_values,
        "labels": batch["label"]
    }

# Add split info for augmentation
trainset = trainset.add_column("__split__", ["train"] * len(trainset))
testset = testset.add_column("__split__", ["test"] * len(testset))

trainset = trainset.map(transform_examples, batched=True, remove_columns=trainset.column_names)
testset = testset.map(transform_examples, batched=True, remove_columns=testset.column_names)

# Model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Data collator
def collate_fn(batch):
    pixel_values = torch.stack([x["pixel_values"] for x in batch])
    labels = torch.tensor([x["labels"] for x in batch])
    return {"pixel_values": pixel_values, "labels": labels}

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./vit-pattern",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,  # Try more epochs for better results
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=8,
    fp16=True,
    remove_unused_columns=True,
    report_to="none",
    learning_rate=3e-5,  # Lower learning rate for better fine-tuning
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset,
    data_collator=collate_fn,
    tokenizer=None,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

# Evaluate
metrics = trainer.evaluate()
print(metrics)

Resolving data files:   0%|          | 0/1900 [00:00<?, ?it/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/950 [00:00<?, ?it/s]

{'loss': 2.9513, 'grad_norm': 1.5533137321472168, 'learning_rate': 2.974736842105263e-05, 'epoch': 0.08}
{'loss': 2.9309, 'grad_norm': 1.49404776096344, 'learning_rate': 2.9494736842105264e-05, 'epoch': 0.17}
{'loss': 2.9204, 'grad_norm': 1.6879453659057617, 'learning_rate': 2.9242105263157893e-05, 'epoch': 0.25}
{'loss': 2.8858, 'grad_norm': 1.7725534439086914, 'learning_rate': 2.8989473684210528e-05, 'epoch': 0.34}
{'loss': 2.8753, 'grad_norm': 1.8019193410873413, 'learning_rate': 2.8736842105263157e-05, 'epoch': 0.42}
{'loss': 2.8683, 'grad_norm': 1.9326295852661133, 'learning_rate': 2.8484210526315792e-05, 'epoch': 0.51}
{'loss': 2.8435, 'grad_norm': 2.039621114730835, 'learning_rate': 2.823157894736842e-05, 'epoch': 0.59}
{'loss': 2.8398, 'grad_norm': 1.8846739530563354, 'learning_rate': 2.7978947368421052e-05, 'epoch': 0.67}
{'loss': 2.8038, 'grad_norm': 2.131182909011841, 'learning_rate': 2.7726315789473684e-05, 'epoch': 0.76}
{'loss': 2.779, 'grad_norm': 1.9602160453796387, 'le

  0%|          | 0/24 [00:00<?, ?it/s]

...

{'eval_loss': 1.1916773319244385, 'eval_accuracy': 0.7315789473684211, 'eval_f1': 0.730497875346291, 'eval_runtime': 53.1814, 'eval_samples_per_second': 7.145, 'eval_steps_per_second': 0.451, 'epoch': 9.0}
{'loss': 0.4589, 'grad_norm': 1.1218756437301636, 'learning_rate': 2.968421052631579e-06, 'epoch': 9.01}
{'loss': 0.4378, 'grad_norm': 1.079724907875061, 'learning_rate': 2.715789473684211e-06, 'epoch': 9.09}
{'loss': 0.4468, 'grad_norm': 1.0606586933135986, 'learning_rate': 2.4631578947368424e-06, 'epoch': 9.18}
{'loss': 0.422, 'grad_norm': 0.9863461256027222, 'learning_rate': 2.2105263157894734e-06, 'epoch': 9.26}
{'loss': 0.435, 'grad_norm': 0.9780309796333313, 'learning_rate': 1.9578947368421052e-06, 'epoch': 9.35}
{'loss': 0.4376, 'grad_norm': 0.9087545871734619, 'learning_rate': 1.7052631578947369e-06, 'epoch': 9.43}
{'loss': 0.437, 'grad_norm': 0.8572819232940674, 'learning_rate': 1.4526315789473685e-06, 'epoch': 9.52}
{'loss': 0.442, 'grad_norm': 1.090192198753357, 'learning_

  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 1.1873637437820435, 'eval_accuracy': 0.7342105263157894, 'eval_f1': 0.7323141919249194, 'eval_runtime': 51.102, 'eval_samples_per_second': 7.436, 'eval_steps_per_second': 0.47, 'epoch': 10.0}
{'train_runtime': 5227.7836, 'train_samples_per_second': 2.908, 'train_steps_per_second': 0.182, 'train_loss': 1.2449252640573603, 'epoch': 10.0}


  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 1.1873637437820435, 'eval_accuracy': 0.7342105263157894, 'eval_f1': 0.7323141919249194, 'eval_runtime': 51.7452, 'eval_samples_per_second': 7.344, 'eval_steps_per_second': 0.464, 'epoch': 10.0}


In [ ]:
from datasets import load_dataset
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
import torch
from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score

# Load dataset and split
dataset = load_dataset("yainage90/fashion-pattern-images")
split = dataset["train"].train_test_split(test_size=0.2, seed=42)
trainset = split["train"]
testset = split["test"]

# Get label info
labels = trainset.features["label"].names
num_labels = len(labels)
id2label = {str(i): l for i, l in enumerate(labels)}
label2id = {l: i for i, l in enumerate(labels)}

# Preprocessing (ViT processor + optional augmentation)
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

augment = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(10),
    # Do NOT include ToTensor or Normalize here
])


def transform_examples(batch):
    # Data augmentation only on training set
    if "train" in batch["__split__"][0]:
        images = [augment(img.convert("RGB")) for img in batch["image"]]
    else:
        images = [img.convert("RGB") for img in batch["image"]]
    # The processor expects PIL Images or numpy arrays in [0, 255] range
    processed = processor(images=images, return_tensors="pt")
    # processed["pixel_values"] is a tensor of shape (batch_size, 3, 224, 224)
    return {
        "pixel_values": processed["pixel_values"],
        "labels": batch["label"]
    }
# Add split info for augmentation
trainset = trainset.add_column("__split__", ["train"] * len(trainset))
testset = testset.add_column("__split__", ["test"] * len(testset))

trainset = trainset.map(transform_examples, batched=True, remove_columns=trainset.column_names)
testset = testset.map(transform_examples, batched=True, remove_columns=testset.column_names)

# Model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


# Data collator
def collate_fn(batch):
    # Ensure pixel_values are tensors and stack them
    pixel_values = torch.stack([x["pixel_values"] if isinstance(x["pixel_values"], torch.Tensor) else torch.tensor(x["pixel_values"]) for x in batch])
    labels = torch.tensor([l for x in batch for l in (x["labels"] if isinstance(x["labels"], list) else [x["labels"]])])
    return {"pixel_values": pixel_values, "labels": labels}
# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./vit2-pattern",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,  # Try more epochs for better results
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=8,
    fp16=True,
    remove_unused_columns=True,
    report_to="none",
    learning_rate=3e-5,  # Lower learning rate for better fine-tuning
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset,
    data_collator=collate_fn,
    tokenizer=None,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

# Evaluate
metrics = trainer.evaluate()
print(metrics)

Resolving data files:   0%|          | 0/1900 [00:00<?, ?it/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/950 [00:00<?, ?it/s]

{'loss': 2.9334, 'grad_norm': 1.889012098312378, 'learning_rate': 2.974736842105263e-05, 'epoch': 0.08}
{'loss': 2.9303, 'grad_norm': 1.8285527229309082, 'learning_rate': 2.9494736842105264e-05, 'epoch': 0.17}
{'loss': 2.8979, 'grad_norm': 1.72324538230896, 'learning_rate': 2.9242105263157893e-05, 'epoch': 0.25}
{'loss': 2.8921, 'grad_norm': 1.7049087285995483, 'learning_rate': 2.8989473684210528e-05, 'epoch': 0.34}
{'loss': 2.8601, 'grad_norm': 1.9082345962524414, 'learning_rate': 2.8736842105263157e-05, 'epoch': 0.42}
{'loss': 2.8629, 'grad_norm': 1.9359761476516724, 'learning_rate': 2.8484210526315792e-05, 'epoch': 0.51}
{'loss': 2.8435, 'grad_norm': 2.0235798358917236, 'learning_rate': 2.823157894736842e-05, 'epoch': 0.59}
{'loss': 2.8228, 'grad_norm': 2.0750834941864014, 'learning_rate': 2.7978947368421052e-05, 'epoch': 0.67}
{'loss': 2.7832, 'grad_norm': 2.2923481464385986, 'learning_rate': 2.7726315789473684e-05, 'epoch': 0.76}
{'loss': 2.7549, 'grad_norm': 2.029365062713623, 'l

  0%|          | 0/24 [00:00<?, ?it/s]

...

{'eval_loss': 1.1650307178497314, 'eval_accuracy': 0.7526315789473684, 'eval_f1': 0.7506950046885792, 'eval_runtime': 51.7999, 'eval_samples_per_second': 7.336, 'eval_steps_per_second': 0.463, 'epoch': 9.0}
{'loss': 0.4488, 'grad_norm': 0.986046552658081, 'learning_rate': 2.968421052631579e-06, 'epoch': 9.01}
{'loss': 0.4256, 'grad_norm': 1.0235873460769653, 'learning_rate': 2.715789473684211e-06, 'epoch': 9.09}
{'loss': 0.4639, 'grad_norm': 1.0451924800872803, 'learning_rate': 2.4631578947368424e-06, 'epoch': 9.18}
{'loss': 0.4194, 'grad_norm': 0.969524085521698, 'learning_rate': 2.2105263157894734e-06, 'epoch': 9.26}
{'loss': 0.4243, 'grad_norm': 0.9470532536506653, 'learning_rate': 1.9578947368421052e-06, 'epoch': 9.35}
{'loss': 0.4226, 'grad_norm': 2.724477529525757, 'learning_rate': 1.7052631578947369e-06, 'epoch': 9.43}
{'loss': 0.4193, 'grad_norm': 0.7828384041786194, 'learning_rate': 1.4526315789473685e-06, 'epoch': 9.52}
{'loss': 0.4427, 'grad_norm': 1.0531713962554932, 'learn

  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 1.15370774269104, 'eval_accuracy': 0.75, 'eval_f1': 0.7489343388324992, 'eval_runtime': 57.2709, 'eval_samples_per_second': 6.635, 'eval_steps_per_second': 0.419, 'epoch': 10.0}
{'train_runtime': 5108.05, 'train_samples_per_second': 2.976, 'train_steps_per_second': 0.186, 'train_loss': 1.2321367283871298, 'epoch': 10.0}


  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 1.15370774269104, 'eval_accuracy': 0.75, 'eval_f1': 0.7489343388324992, 'eval_runtime': 56.1626, 'eval_samples_per_second': 6.766, 'eval_steps_per_second': 0.427, 'epoch': 10.0}


In [45]:
from datasets import load_dataset
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
import torch
from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score

# Load dataset and split
dataset = load_dataset("yainage90/fashion-pattern-images")
split = dataset["train"].train_test_split(test_size=0.2, seed=42)
trainset = split["train"]
testset = split["test"]

# Get label info
labels = trainset.features["label"].names
num_labels = len(labels)
id2label = {str(i): l for i, l in enumerate(labels)}
label2id = {l: i for i, l in enumerate(labels)}

# Preprocessing (ViT processor + optional augmentation)
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

augment = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(10),
    # Do NOT include ToTensor or Normalize here
])


def transform_examples(batch):
    # Data augmentation only on training set
    if "train" in batch["__split__"][0]:
        images = [augment(img.convert("RGB")) for img in batch["image"]]
    else:
        images = [img.convert("RGB") for img in batch["image"]]
    # The processor expects PIL Images or numpy arrays in [0, 255] range
    processed = processor(images=images, return_tensors="pt")
    # processed["pixel_values"] is a tensor of shape (batch_size, 3, 224, 224)
    return {
        "pixel_values": processed["pixel_values"],
        "labels": batch["label"]
    }
# Add split info for augmentation
trainset = trainset.add_column("__split__", ["train"] * len(trainset))
testset = testset.add_column("__split__", ["test"] * len(testset))

trainset = trainset.map(transform_examples, batched=True, remove_columns=trainset.column_names)
testset = testset.map(transform_examples, batched=True, remove_columns=testset.column_names)

# Model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


# Data collator
def collate_fn(batch):
    # Ensure pixel_values are tensors and stack them
    pixel_values = torch.stack([x["pixel_values"] if isinstance(x["pixel_values"], torch.Tensor) else torch.tensor(x["pixel_values"]) for x in batch])
    labels = torch.tensor([l for x in batch for l in (x["labels"] if isinstance(x["labels"], list) else [x["labels"]])])
    return {"pixel_values": pixel_values, "labels": labels}
# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./vit2-pattern",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,  # Try more epochs for better results
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=8,
    fp16=True,
    remove_unused_columns=True,
    report_to="none",
    learning_rate=3e-5,  # Lower learning rate for better fine-tuning
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset,
    data_collator=collate_fn,
    tokenizer=None,
    compute_metrics=compute_metrics,
)

# Train
trainer.train(resume_from_checkpoint="./vit2-pattern/checkpoint-950")

# Evaluate
metrics = trainer.evaluate()
print(metrics)

Resolving data files:   0%|          | 0/1900 [00:00<?, ?it/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for mor

  0%|          | 0/1425 [00:00<?, ?it/s]

c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\trainer.py:2833: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = to

{'loss': 0.4204, 'grad_norm': 0.9561194181442261, 'learning_rate': 9.957894736842106e-06, 'epoch': 10.02}
{'loss': 0.4244, 'grad_norm': 0.9059658050537109, 'learning_rate': 9.789473684210527e-06, 'epoch': 10.11}
{'loss': 0.4234, 'grad_norm': 0.9929202795028687, 'learning_rate': 9.621052631578949e-06, 'epoch': 10.19}
{'loss': 0.4175, 'grad_norm': 1.1322929859161377, 'learning_rate': 9.45263157894737e-06, 'epoch': 10.27}
{'loss': 0.3923, 'grad_norm': 0.9534863829612732, 'learning_rate': 9.28421052631579e-06, 'epoch': 10.36}
{'loss': 0.3972, 'grad_norm': 0.780124306678772, 'learning_rate': 9.11578947368421e-06, 'epoch': 10.44}
{'loss': 0.4063, 'grad_norm': 1.0266274213790894, 'learning_rate': 8.947368421052632e-06, 'epoch': 10.53}
{'loss': 0.3943, 'grad_norm': 0.9619603753089905, 'learning_rate': 8.778947368421053e-06, 'epoch': 10.61}
{'loss': 0.3782, 'grad_norm': 0.8985133767127991, 'learning_rate': 8.610526315789474e-06, 'epoch': 10.69}
{'loss': 0.3768, 'grad_norm': 0.8068420886993408, 

...

{'eval_loss': 1.072715401649475, 'eval_accuracy': 0.7368421052631579, 'eval_f1': 0.7375867046891705, 'eval_runtime': 77.58, 'eval_samples_per_second': 4.898, 'eval_steps_per_second': 0.309, 'epoch': 14.0}
{'loss': 0.2844, 'grad_norm': 0.6431567668914795, 'learning_rate': 1.8736842105263158e-06, 'epoch': 14.06}
{'loss': 0.2756, 'grad_norm': 0.5587624907493591, 'learning_rate': 1.7052631578947369e-06, 'epoch': 14.15}
{'loss': 0.2788, 'grad_norm': 0.5767632126808167, 'learning_rate': 1.5368421052631579e-06, 'epoch': 14.23}
{'loss': 0.2837, 'grad_norm': 0.7035180330276489, 'learning_rate': 1.3684210526315791e-06, 'epoch': 14.32}
{'loss': 0.2839, 'grad_norm': 0.6471250653266907, 'learning_rate': 1.2000000000000002e-06, 'epoch': 14.4}
{'loss': 0.2806, 'grad_norm': 0.7250261902809143, 'learning_rate': 1.031578947368421e-06, 'epoch': 14.48}
{'loss': 0.2801, 'grad_norm': 0.6492757797241211, 'learning_rate': 8.631578947368421e-07, 'epoch': 14.57}
{'loss': 0.2747, 'grad_norm': 0.7524358034133911,

  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 1.0711021423339844, 'eval_accuracy': 0.7368421052631579, 'eval_f1': 0.7375867046891705, 'eval_runtime': 131.5153, 'eval_samples_per_second': 2.889, 'eval_steps_per_second': 0.182, 'epoch': 15.0}
{'train_runtime': 2194.7286, 'train_samples_per_second': 10.389, 'train_steps_per_second': 0.649, 'train_loss': 0.10791874935752467, 'epoch': 15.0}


  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 1.0711021423339844, 'eval_accuracy': 0.7368421052631579, 'eval_f1': 0.7375867046891705, 'eval_runtime': 91.4089, 'eval_samples_per_second': 4.157, 'eval_steps_per_second': 0.263, 'epoch': 15.0}


In [47]:
# Save the model
trainer.save_model("./vit2-pattern-final")
# Save the processor
processor.save_pretrained("./vit2-pattern-final")


['./vit2-pattern-final\\preprocessor_config.json']

In [ ]:
image_path = ""